D48 data-science-london-scikit-learn  
透過 Kaggle 的實戰練習，把資料下載下來，自己做訓練集/測試集的切分，送進模型訓練，評估結果，再把測試集的答案上傳到 Kaggle 得到最終成績。  

請完成 [scikit-learn-practice](https://www.kaggle.com/c/data-science-london-scikit-learn) 比賽(點擊連結可至競賽頁面)，讓大家熟悉 Scikit-learn 的比賽  

i.總共有一千筆訓練資料、40個 features，二元分類問題，練習 features scaling、建模、調參數等步驟  
ii.每天最多上傳 10 次結果  
iii.請在 private / public leaderboard 上取得 0.7 以上的準確率  
iv.可多參考別人的 Kernel，學習別人的寫法與思路，完成自己的 Kaggle 競賽  

In [19]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [20]:
data_path = 'data-science-london-scikit-learn/'
test = pd.read_csv(data_path + "test.csv",header=None)
fea_all = pd.read_csv(data_path + "train.csv",header=None)
label_all = pd.read_csv(data_path + "trainLabels.csv",header=None)

In [21]:
fea_train, fea_test, label_train, label_test = train_test_split(fea_all, label_all, test_size=0.1, random_state=1)
gbc = GradientBoostingClassifier()
gbc.fit(fea_train, label_train)
#pred
org_pred = gbc.predict(fea_test)
#accuracy
print(f"Accuracy : {accuracy_score(org_pred, label_test)}")

Accuracy : 0.95


In [22]:
#dict of hyper-parameter
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

#search best parameter
#create search object
para_search = GridSearchCV(gbc, param_grid, n_jobs=-1, verbose=1)
#start search
search_result = para_search.fit(fea_train, label_train)
print(f"Best accuracy : {search_result.best_score_}, by using parameters : {search_result.best_params_}")

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.9s finished
Best accuracy : 0.8677777777777778, by using parameters : {'max_depth': 5, 'n_estimators': 200}


In [23]:
gbc_bestpara = GradientBoostingClassifier(n_estimators=search_result.best_params_['n_estimators'], max_depth=search_result.best_params_['max_depth'])
#train model
gbc_bestpara.fit(fea_train, label_train)
#pred
bestpara_pred = gbc_bestpara.predict(fea_test)
#accuracy
print(f"Accuracy : {accuracy_score(bestpara_pred, label_test)}")

Accuracy : 0.96


In [24]:
#train model with all train data
gbc_bestpara.fit(fea_all, label_all)
test_pred = gbc_bestpara.predict(test)

In [27]:
test_pred.shape

(9000,)

In [38]:
submit = pd.DataFrame(test_pred)
submit.columns = ["Solution"] #column name = Solution
submit["Id"] = np.arange(1, submit.shape[0]+1) #add column "Id"
submit = submit[["Id", "Solution"]] #move Column "Id" to 1st col
submit.head()

,Id,Solution
0,1,1
1,2,0
2,3,1
3,4,0
4,5,0


In [39]:
submit.to_csv("London.csv", index=False)